In [1]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision
from scipy import special as S
from scipy.stats import entropy
from scipy.spatial import distance
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# Meitner Machine
import ewm
import utils
from dataloader import BottleLoader

In [3]:
device = torch.device(1)

In [4]:
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

In [5]:
mlp = False
if mlp:
    EWM_root = exp_root + 'ewm_models/mlp_ewm/'
else:
    EWM_root = exp_root + 'ewm_models/conv_ewm/'

In [6]:
# Path to model weights
weights_dir = "weights/"

In [7]:
EWM_paths = []
for path in os.listdir(EWM_root):
    EWM_paths.append(os.path.join(EWM_root, path))

print("-"*60)
for i in range(len(EWM_paths)):
    EWM_name = EWM_paths[i].split('/')[-1]
    print("\n Exp_{}:".format(str(i)), EWM_name, '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: ewm_64_192 

------------------------------------------------------------

 Exp_1: 02-21-2020_10-40-28_ewm_20_epochs_Code_Vectors_64_384 

------------------------------------------------------------

 Exp_2: ewm_128_320 

------------------------------------------------------------

 Exp_3: ewm_nonsense2_64-384 

------------------------------------------------------------

 Exp_4: ewm_nonsense_64_384 

------------------------------------------------------------

 Exp_5: ewm_256_6528 

------------------------------------------------------------

 Exp_6: evaluated_models 

------------------------------------------------------------

 Exp_7: ewm_256_1632 

------------------------------------------------------------

 Exp_8: ewm_nonsense3_64-384 

------------------------------------------------------------


In [8]:
EWM_dir = EWM_paths[1]

In [9]:
# Create the full path to the EWM model
EWM_path = os.path.join(EWM_root, EWM_dir) + "/"
print("Path to EWM Generator Model set as: \n{}".format(EWM_path))

Path to EWM Generator Model set as: 
/media/hdd1/kai/particle_generator/experiments/ewm_models/conv_ewm/02-21-2020_10-40-28_ewm_20_epochs_Code_Vectors_64_384/


In [10]:
config_csv = EWM_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [11]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
n_hidden = int(config_df[config_df['Unnamed: 0'].str.contains("n_hidden")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())
z_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("z_dim")==True]['0'].values.item())
print("{} Layer model with {} hidden units per layer".format(n_layers, n_hidden))

4 Layer model with 512 hidden units per layer


In [12]:
# Model kwargs
fc_sizes = [n_hidden] * n_layers
ewm_kwargs = { 'z_dim':z_dim, 'fc_sizes':fc_sizes, 'n_out':l_dim }

In [13]:
# Create generator on GPU
G = ewm.ewm_G(**ewm_kwargs).to(device)
G.weights_init()

In [14]:
# Get checkpoint name(s)
EWM_checkpoint_path  = EWM_path + weights_dir
EWM_checkpoint_names = []
for file in os.listdir(EWM_checkpoint_path):
    EWM_checkpoint_names.append(os.path.join(EWM_checkpoint_path, file))

In [15]:
print("-"*60)
for i in range(len(EWM_checkpoint_names)):
    name = EWM_checkpoint_names[i].split('/')[-1]
    print("\n {} :".format(str(i)), name, '\n')
    print("-"*60)

------------------------------------------------------------

 0 : best_ewm_ep_0.tar 

------------------------------------------------------------

 1 : best_ewm_ep_19.tar 

------------------------------------------------------------


In [16]:
# Select the checkpoint you want
EWM_checkpoint = EWM_checkpoint_names[1]

In [17]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(EWM_checkpoint)

In [18]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
G.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [19]:
# Put the model in evaluation mode
G.eval()

ewm_G(
  (fc): Sequential(
    (0): Sequential(
      (0): Linear(in_features=100, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.2)
    )
  )
  (out): Sequential(
    (0): Linear(in_features=512, out_features=384, bias=True)
  )
)

In [20]:
config = {'ewm_target':'conv', 'num_workers':0, 'batch_size':1, 'shuffle':False, 'drop_last':False,
          'vec_root':'/media/hdd1/kai/particle_generator/code_vectors_test/', 'dataset':64, 'l_dim':l_dim}

In [21]:
test_loader = utils.get_test_loader(config)

Code-Target examples will be loaded from subfolder of: 
/media/hdd1/kai/particle_generator/code_vectors_test/conv_ae/code_vectors_64_384/


In [22]:
g_vectors = []; t_vectors = []; epsilon = 1e-6

In [23]:
for _, test_vec in enumerate(test_loader):
    
    # Get output from the generator model
    z_rand = torch.randn(config['batch_size'], 1, 100).to(device)
    code_vec = G(z_rand).view(-1).to(device)
    g_vectors.append(code_vec.detach().cpu().numpy())
    
    # Keep test vec
    test_vec = test_vec.view(-1).detach().cpu().numpy()
    t_vectors.append(test_vec)
    
    del code_vec
    del test_vec

[SciPi Entropy Function](http://scipy.github.io/devdocs/generated/scipy.stats.entropy.html)

In [24]:
g_vectors = np.array([g for g in g_vectors]).flatten() + epsilon
t_vectors = np.array([t for t in t_vectors]).flatten() + epsilon

In [47]:
len(np.nonzero(g_vectors)[0]) == len(g_vectors)

True

In [48]:
len(np.nonzero(t_vectors)[0]) == len(t_vectors)

True

In [37]:
sum(_p * np.log(_p / _q) for _p, _q in zip(g_vectors, t_vectors) if _p != 0)

/home/kseuro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [26]:
ent = entropy(pk=t_vectors, qk=g_vectors)

In [27]:
ent

inf

In [28]:
len(np.nonzero(g_vectors)[0])

3840000

In [29]:
kl_div = S.kl_div(g_vectors, t_vectors)

In [30]:
kl_div

array([inf, inf, inf, ..., inf, inf, inf], dtype=float32)

In [31]:
g_vectors - t_vectors

array([-0.01137161,  0.07320309,  0.0429039 , ...,  0.3619461 ,
        0.16585541,  0.04455185], dtype=float32)

In [32]:
dist = distance.jensenshannon(t_vectors, g_vectors)

In [33]:
dist

inf